# LSTM
RNN在处理长期依赖（时间序列上距离较远的节点）时会遇到巨大的困难，因为计算距离较远的节点之间的联系时会涉及雅可比矩阵的多次相乘，会造成梯度消失或者梯度膨胀的现象。

循环神经网络除了训练困难，还有一个更严重的问题，那就是短时记忆(Short-term memory).它在处理较长的句子时，往往只能够理解有限长度内的信息，而对于位于较长范围类的有用信息往往不能够很好的利用起来。

![](./images/LSTM2.png)

## 原理

RNN的核心思想是上一个时间戳的状态向量$h_{t-1}$与当前时间戳的输入$x_t$经过线性变换后, 通过激活函数得到新的状态向量$h_{t}$.
LSTM 新增了一个状态向量$𝑪$, 同时引入了门控(Gate)机制，通过门控单元来控制信息的遗忘和刷新，他们包含一个 sigmoid 神经网络层和一个 pointwise 乘法操作.

### 遗忘层门

作用: 将细胞状态中的信息选择性的遗忘, 作用于 LSTM 状态向量𝒄上面，.

操作步骤：该门会读取$h_{t-1}$和$x_t$，输出一个在 0 到 1 之间的数值给每个在细胞状态$C_{t-1}$中的数字。1 表示“完全保留”，0 表示“完全舍弃”。

公式: $$f_t = \sigma(W_f[h_{t-1}, x_t] + b_f)$$

### 输入层门

作用: 将新的信息选择性的记录到细胞状态中, 控制LSTM对输入的接收程度

操作步骤: 

1. sigmoid层决定什么值我们将要更新($i_t$);   
2. tanh 层创建一个新的候选值向量$\tilde{C}_t​$加入到状态中;  
3. 将$c_{t-1}$更新为$c_{t}$, 丢弃需要丢弃的旧状态, 获得需要获得的新状态

公式:
$$
i_t = \sigma(W_i[h_{t-1}, x_t] + b_i) \\
\tilde{C}_t = tanh(W_C[h_{t-1}, x_t] + b_C) \\
C_t = f_t * C_{t-1} + i_t * \tilde C_t
$$


### 输出门层

作用: 确定输出什么值, 内部状态$C_t$不会直接用于输出

操作步骤: 

1. 通过sigmoid 层来确定细胞状态的哪个部分将输出
2. 把细胞状态通过 tanh 进行处理，并将它和 sigmoid 门的输出相乘, 输出确定输出的部分

公式:
$$
o_t = \sigma(W_o[h_{t-1}, x_t] + b_o) \\
h_t = o_t * tanh(C_t)
$$


|输入门控 |遗忘门控 |LSTM行为|
|---|---|---|
|0|1|只是用记忆|
|1|1| 综合输入和记忆|
|0|0|清零记忆|
|1|0|输入覆盖记忆|


### LSTMCell

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
from tensorflow.keras import layers, Input, Model, Sequential

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        print(gpu)
except RuntimeError as e:
    print(e)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [6]:
x = tf.random.normal([2, 80, 100])  # 
xt = x[:, 0, :]  # 第一个单词, 第一个时间戳的输入

cell = layers.LSTMCell(64)  # 与SimpleRNNCell 类似
state = [tf.zeros([2, 64]), tf.zeros([2, 64])]  # 初始状态h0
out, state = cell(xt, state)

In [8]:
cell.get_config()

{'name': 'lstm_cell',
 'trainable': True,
 'dtype': 'float32',
 'units': 64,
 'activation': 'tanh',
 'recurrent_activation': 'sigmoid',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'unit_forget_bias': True,
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0,
 'implementation': 2}